<center><h1>Testing Notebook</h1></center>


## Loading Entities:

 - Power System
 - Ant Colony

In [ ]:
from acopoweropt import system, colony

PSystem = system.PowerSystem(name='s10')
Colony = colony.Colony(n_ants=5,phr_evp_rate=0.25)

## Testing Power System

In [ ]:
# Randomly select an operation
operation = PSystem.sample_operation()

# Solve system
solution = PSystem.solve(operation=operation)

In [ ]:
solution.get('Ft')

## Testing Colony

In [1]:
import pandas as pd
import numpy as np

from acopoweropt import colony, system

In [31]:
import pandas as pd
import numpy as np
import random

from acopoweropt import colony, system

PSystem = system.PowerSystem(name='s10')
Colony = colony.PowerColony(n_ants=25,
                            pheromone_evap_rate=0.25,
                            PowerSystem=PSystem)

print("Iteracao: {} - MinValue: {}".format(0, Colony.paths[0].distance.min()))

n_iter = 150

for i in range(n_iter):
    # Getting last taken paths
    paths = Colony.paths[i].copy()
    
    # Evaporate Pheromone
    Colony.pheromone = (1 - Colony.pheromone_evap_rate) * Colony.pheromone
   
    # Selecting Ants (80/20) to follow or not the pheromone paths
    l = []
    for ant in range(1, Colony.n_ants + 1):
        decision = random.random()
        if decision <= 0.8:
            ## ANT Follows Path
            operative_zones = Colony.choose_path()
            chosen_path = ",".join([str(int) for int in operative_zones])
            existing_path = paths.query("path == '{}'".format(chosen_path)) 

            ## Check if chosen path was already calculated:
            if existing_path.values.size != 0:
                existing_path = existing_path.reset_index()
                existing_path['ant'] = ant
                existing_path = existing_path.set_index('ant')
                l.append(existing_path)
            else:
                operation = PSystem.get_operation(operative_zones=operative_zones)
                solution = pd.DataFrame([Colony.seek_food(ant=ant, operation=operation, PowerSystem=PSystem)]).set_index('ant')
                l.append(solution)
                
        else:
        ## ANT Sets to new path
            operation = PSystem.sample_operation()
            solution = pd.DataFrame([Colony.seek_food(ant=ant, operation=operation, PowerSystem=PSystem)]).set_index('ant')
            l.append(solution)

    new_paths = pd.concat(l).sort_index()
    Colony.paths.update({i+1: new_paths})
    
    # Updating Pheromone
    new_pheromone = Colony.update_pheromone(paths=new_paths)
    Colony.pheromone_history.update({i+1: new_pheromone})
    
    print("Iteracao: {} - MinValue: {}".format(i+1, Colony.paths[i+1].distance.min()))

Iteracao: 0 - MinValue: 670.3805877176532
Iteracao: 1 - MinValue: 690.1419599895979
Iteracao: 2 - MinValue: 669.7456153149215
Iteracao: 3 - MinValue: 675.0931096808683
Iteracao: 4 - MinValue: 683.6539626404996
Iteracao: 5 - MinValue: 678.9758515900681
Iteracao: 6 - MinValue: 692.6708337127171
Iteracao: 7 - MinValue: 689.877287338153
Iteracao: 8 - MinValue: 665.9906264854745
Iteracao: 9 - MinValue: 684.9399950472725
Iteracao: 10 - MinValue: 677.9772117138508
Iteracao: 11 - MinValue: 670.7968707059312
Iteracao: 12 - MinValue: 684.9400112908087
Iteracao: 13 - MinValue: 684.5566586837862
Iteracao: 14 - MinValue: 671.3431363286034
Iteracao: 15 - MinValue: 690.8050632843031
Iteracao: 16 - MinValue: 678.0756918211873
Iteracao: 17 - MinValue: 684.3654121563842
Iteracao: 18 - MinValue: 664.2099125627253
Iteracao: 19 - MinValue: 670.4620269524241
Iteracao: 20 - MinValue: 678.5641039362067
Iteracao: 21 - MinValue: 670.1184669809858
Iteracao: 22 - MinValue: 662.9383400414536
Iteracao: 23 - MinValu

In [32]:
l = []
for path in Colony.paths:
    l.append(Colony.paths[path])
df = pd.concat(l)

In [34]:
Colony.pheromone

,1,2,3,4,5,6,7,8,9,10
opz,,,,,,,,,,
1,66.292709,49.881150,31.703668,40.644173,49.899381,35.720817,46.459426,33.953738,22.408411,41.496623
2,56.586960,26.612022,39.430926,43.546449,29.767479,29.929896,52.547847,30.345202,60.883114,36.300803
3,0.000000,46.386497,51.745075,38.689046,43.212808,57.228956,23.872395,58.580729,39.588143,45.082242


In [33]:
df.groupby(['path']).count().sort_values(by=['distance'], ascending=False)

,status,distance,tau
path,,,
"2,3,3,2,3,3,3,2,3,3",3,3,3
"1,1,2,2,2,3,3,3,2,2",3,3,3
"2,3,2,3,1,3,2,3,3,2",3,3,3
"1,2,3,3,1,3,1,1,3,1",3,3,3
"2,3,2,3,1,3,3,3,3,1",3,3,3
...,...,...,...
"1,3,1,2,2,1,1,1,1,3",1,1,1
"1,3,1,2,2,1,1,3,1,1",1,1,1
"1,3,1,2,2,1,1,3,1,2",1,1,1


In [4]:
optimal = [1,3,1,1,3,4,1,1,1,1,1,2,1,1,1]
test = [1,2,1,1,2,3,1,1,1,1,1,2,1,1,1]

operation = PSystem.get_operation(operative_zones=test)

solution = PSystem.solve(operation=operation)

pd.DataFrame([Colony.seek_food(ant=1, operation=operation, PowerSystem=PSystem)]).set_index('ant')

,path,status,distance,tau
ant,,,,
1,"1,2,1,1,2,3,1,1,1,1,1,2,1,1,1",optimal,32702.10893,0.000031


In [5]:
solution['operation']

,opz,a,b,c,Pmin,Pmax,Pg,Fi
tgu,,,,,,,,
1,1,671.03,10.07,0.000299,150,455,454.999922,5314.779665
2,2,574.54,10.22,0.000183,225,305,304.999927,3708.662817
3,1,374.59,8.8,0.001126,20,130,129.999959,1537.619026
4,1,374.59,8.8,0.001126,20,130,129.999959,1537.619026
5,2,461.37,10.4,0.000205,200,260,259.999919,3179.227147
6,3,630.14,10.1,0.000301,395,430,429.999920,5028.794073
7,1,548.2,9.87,0.000364,135,465,464.999927,5216.455158
8,1,227.09,11.5,0.000338,60,300,60.005308,918.368057
9,1,173.72,11.21,0.000807,25,162,109.764614,1413.904274


In [28]:
solution['operation']['b']

tgu
1     10.07
2     10.22
3       8.8
4       8.8
5      10.4
6      10.1
7      9.87
8      11.5
9     11.21
10    10.72
11    11.21
12      9.9
13    13.12
14    12.12
15    12.41
Name: b, dtype: object

In [ ]:
.size == 0

In [50]:
paths.query("path == '{}'".format(chosen_path)).values.size == 0

True

In [47]:
Colony.paths

{0:                               path   status      distance       tau
 ant                                                                
 1    1,2,1,1,2,4,1,1,1,1,1,2,1,1,1  optimal  32672.124337  0.000031
 2    1,1,1,1,4,4,1,1,1,1,1,3,1,1,1  optimal  32635.669146  0.000031
 3    1,3,1,1,4,2,1,1,1,1,1,1,1,1,1  optimal  32575.017052  0.000031
 4    1,4,1,1,2,4,1,1,1,1,1,2,1,1,1  optimal  32558.357929  0.000031
 5    1,3,1,1,1,3,1,1,1,1,1,1,1,1,1  optimal  32688.619792  0.000031}

In [209]:
[Colony.pheromone.sample(n=1, weights=Colony.pheromone[tgu], axis=0).index[0] for tgu in Colony.pheromone.columns]

[1, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]

[1, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]

In [161]:
Colony.pheromone.sample(n=1, weights=Colony.pheromone[2], axis=0).index[0]

4

In [165]:
Colony.pheromone

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
opz,,,,,,,,,,,,,,,
1,0.3066,0.0000,0.3066,0.3066,0.0000,0.0612,0.3066,0.3066,0.3066,0.3066,0.3066,0.0612,0.3066,0.3066,0.3066
2,0.0000,0.0000,0.0000,0.0000,0.1840,0.0612,0.0000,0.0000,0.0000,0.0000,0.0000,0.1226,0.0000,0.0000,0.0000
3,0.0000,0.1226,0.0000,0.0000,0.1226,0.1842,0.0000,0.0000,0.0000,0.0000,0.0000,0.1228,0.0000,0.0000,0.0000
4,0.0000,0.1840,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [66]:
import random


random.choices(population=Colony.pheromone[1], weights=[Colony.pheromone[1][i+1]/Colony.pheromone[1].sum() for i,n in enumerate(Colony.pheromone[1])],k=1)

[1]

In [75]:
[Colony.pheromone[1][i+1]/Colony.pheromone[1].sum() for i,n in enumerate(Colony.pheromone[1])]

[1.0, 0.0, 0.0, 0.0]

In [69]:
Colony.pheromone[1].sum()

0.3066